In [2]:
import requests
import io
import os
import numpy as np
import sys
import matplotlib.pyplot as plt
import tensorflow as tf            
plt.rcParams['figure.figsize'] = (16, 9)  #ver graficas grandes 
plt.style.use('ggplot')
tf.device('/cpu:0')

In [3]:
tf.device('/cpu:0')

In [4]:
fileUrl     ='https://raw.githubusercontent.com/DuvanAndresVelasquez/DeepLearning/main/ciudad_perros.txt'
fileContent = tf.keras.utils.get_file('ciudad_perros.txt',fileUrl)
texto       = open(fileContent, 'rb').read().decode(encoding='utf-8')
raw_text    = texto.lower()
print(raw_text)

40960/28367 [===========================================] - 0s 0us/step
-cuatro -dijo el jaguar.
los rostros se suavizaron en el resplandor vacilante que el globo de luz difundía por el recinto, a través
de escasas partículas limpias de vidrio: el peligro había desaparecido para todos, salvo para porfirio
cava. los dados estaban quietos, marcaban tres y uno, su blancura contrastaba con el suelo sucio.
-cuatro -repitió el jaguar- ¿quién?
-yo -murmuró cava- dije cuatro.
-apúrate -replicó el jaguar- ya sabes, el segundo de la izquierda.
cava sintió frío. los baños estaban al fondo de las cuadras, separados de ellas por una delgada puerta de
madera, y no tenían ventanas. en años anteriores, el invierno sólo llegaba al dormitorio de los cadetes,
colándose por los vidrios rotos y las rendijas; pero este año era agresivo y casi ningún rincón de] colegio
se libraba del viento, que, en las noches, conseguía penetrar hasta en los baños, disipar la hediondez
acumulada durante el día y destruir su

In [5]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
print(char_to_int)

['\n', '\r', ' ', '!', '"', "'", '(', ',', '-', '.', '6', ':', ';', '?', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '“', '”']
{'\n': 0, '\r': 1, ' ': 2, '!': 3, '"': 4, "'": 5, '(': 6, ',': 7, '-': 8, '.': 9, '6': 10, ':': 11, ';': 12, '?': 13, ']': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'j': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'q': 30, 'r': 31, 's': 32, 't': 33, 'u': 34, 'v': 35, 'x': 36, 'y': 37, 'z': 38, '¡': 39, '¿': 40, 'á': 41, 'é': 42, 'í': 43, 'ñ': 44, 'ó': 45, 'ú': 46, 'ü': 47, '“': 48, '”': 49}


In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))

En total hay 27733 caracteres, y el diccionario tiene un tamaño de 50 caracteres.


In [7]:
seq_length = 50
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	print("Seq(",i,")=",raw_text[i:i + seq_length],"---->",raw_text[i + seq_length])
	seq_in 	= raw_text[i:i + seq_length]         					
	seq_out = raw_text[i + seq_length]										
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Se generaron ",format(n_patterns) ," secuencias texto de un tamaño de ",seq_length," caracteres" )
print("Como se ven los datos de X convertidos a números\n")
print(dataX)
print("\nComo se ven los datos de Y convertidos a números\n")
print(dataY)

Se han truncado las últimas 5000 líneas del flujo de salida.
Seq( 24509 )= e me fíe, te ofrezco cartas y novelitas, de veras  ----> q
Seq( 24510 )=  me fíe, te ofrezco cartas y novelitas, de veras q ----> u
Seq( 24511 )= me fíe, te ofrezco cartas y novelitas, de veras qu ----> e
Seq( 24512 )= e fíe, te ofrezco cartas y novelitas, de veras que ---->  
Seq( 24513 )=  fíe, te ofrezco cartas y novelitas, de veras que  ----> e
Seq( 24514 )= fíe, te ofrezco cartas y novelitas, de veras que e ----> n
Seq( 24515 )= íe, te ofrezco cartas y novelitas, de veras que en ---->  
Seq( 24516 )= e, te ofrezco cartas y novelitas, de veras que en  ----> l
Seq( 24517 )= , te ofrezco cartas y novelitas, de veras que en l ----> o
Seq( 24518 )=  te ofrezco cartas y novelitas, de veras que en lo ----> s
Seq( 24519 )= te ofrezco cartas y novelitas, de veras que en los ---->  
Seq( 24520 )= e ofrezco cartas y novelitas, de veras que en los  ----> t
Seq( 24521 )=  ofrezco cartas y novelitas, de veras que en los 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
X = np.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = tf.keras.utils.to_categorical(dataY)

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))    #creamos una capa con 256 unidades de memoria
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))            #Softmax convierte un vector de valores en una distribución de probabilidad para cada uno de los 39
                                                                              #Softmax se utiliza a menudo como la activación para la última capa de una red de clasificación
#utilizamos el algoritmo de optimización de ADAM para la velocidad
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
filepath="checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
history = model.fit(X, y, epochs=1000, batch_size=128, verbose=1,callbacks=callbacks_list)

Epoch 1/1000
216/217 [============================>.] - ETA: 0s - loss: 3.0851
Epoch 00001: loss improved from inf to 3.08491, saving model to checkpoints/weights-improvement-01-3.0849.hdf5
217/217 [==============================] - 11s 21ms/step - loss: 3.0849
Epoch 2/1000
217/217 [==============================] - ETA: 0s - loss: 3.0440
Epoch 00002: loss improved from 3.08491 to 3.04404, saving model to checkpoints/weights-improvement-02-3.0440.hdf5
217/217 [==============================] - 4s 20ms/step - loss: 3.0440
Epoch 3/1000
217/217 [==============================] - ETA: 0s - loss: 3.0130
Epoch 00003: loss improved from 3.04404 to 3.01302, saving model to checkpoints/weights-improvement-03-3.0130.hdf5
217/217 [==============================] - 4s 20ms/step - loss: 3.0130
Epoch 4/1000
217/217 [==============================] - ETA: 0s - loss: 2.9405
Epoch 00004: loss improved from 3.01302 to 2.94053, saving model to checkpoints/weights-improvement-04-2.9405.hdf5
217/217 [=====

In [14]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [15]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de 10000 caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
 
 l o s   b a ñ o s"
 de otras cuadras, atravesando ruidosamante los dormitorios, a propósito, pero en ninguno se

del baño, se detiene. alguien desvaría: distingue apenas, entre un río de palabras confusas, un nombre

enseñaba las cartas y las fotos que recibía, y me contaba sus penas, escríbele bonito que la quiero mucho,
yo no soy un cura, qué carajo, usted es un tarado. ¿lidia?" en la séptima sección, junto a los urinarios,

tirados en el suelo y otro apoyado en la pared. la puerta del baño está abierta y alberto los distingue a lo
lejos, desde el umbral de la cuadra. avanza, lo intercepta una sombra.
-¿qué hay? ¿quién es?
- el coronel. ¿tienen permiso para timbear? el servicio no se aband
Done.


In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(256))                                                                #agregaremos una segunda capa. 
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam')
#cambiamos el nombre de archivo de los pesos con puntos de control para que 
#podamos distinguir entre los pesos de esta red 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

In [17]:
#los tiempos de entrenamiento aumentaran al doble que en la versión anterior
model.fit(X, y, epochs=50, batch_size=32, callbacks=callbacks_list)

Epoch 1/50
865/866 [============================>.] - ETA: 0s - loss: 3.0539
Epoch 00001: loss did not improve from 0.06850
866/866 [==============================] - 31s 32ms/step - loss: 3.0538
Epoch 2/50
865/866 [============================>.] - ETA: 0s - loss: 2.8530
Epoch 00002: loss did not improve from 0.06850
866/866 [==============================] - 27s 32ms/step - loss: 2.8529
Epoch 3/50
865/866 [============================>.] - ETA: 0s - loss: 2.7410
Epoch 00003: loss did not improve from 0.06850
866/866 [==============================] - 27s 32ms/step - loss: 2.7410
Epoch 4/50
865/866 [============================>.] - ETA: 0s - loss: 2.6678
Epoch 00004: loss did not improve from 0.06850
866/866 [==============================] - 27s 32ms/step - loss: 2.6678
Epoch 5/50
865/866 [============================>.] - ETA: 0s - loss: 2.5971
Epoch 00005: loss did not improve from 0.06850
866/866 [==============================] - 28s 32ms/step - loss: 2.5971
Epoch 6/50
865/866 [

In [18]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"n t e :   -   e l   c o r o n e l   d i j o   u n a   v e z   q u e   p o d í a m o s   c o n s u l"
en la quinta sección, poco antes, un no oe estivo. mueeno decir de la cabeza, no del cuerpo ce 

el ueriear ee jorala es pu hadra si pobo el eoriocdo, al fin, el coche se inmoviliza distingue a lo lejos de la preseón del fusil, que me jaranto a los pírio pueserra en la silta se desuvoó cl
pieurro a la pista de desfile, que separaba la nuerta y penóa al satio y desde estoy lue eutmar lueer. senas almá. vin senascca a la credra, se mantuva juntas, en la oscuridad- ¡maditoa al héroe mererta a la presta del horizo de la prevención, ja camado de valtano en el suelo y el patio 
de un cuerpo y qu madre se besaban en la lucuridad del descampado. 
un movimiento pera qui le los tiraba de la dintura; parece más frágels y peqoeño, senbr uiinte soles y me hismectan 
c vl mespo de dadetes, avrásas a cimo eo maguar.
- no había nadie, lus aatos  lmsiónitado en la pier
Done.


In [19]:
!pip install h5py

In [20]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modeloRNN_cuentosPesos.hdf5")
model.save('modelRNN_cuentos_v_h5.h5')
print("modelo salvado en disco")

modelo salvado en disco


In [24]:
chars = sorted(list(set("fusil")))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))
pattern = dataX[5]
print(pattern)

En total hay 27733 caracteres, y el diccionario tiene un tamaño de 5 caracteres.
[31, 28, 2, 8, 18, 23, 24, 28, 2, 19, 25, 2, 24, 15, 21, 34, 15, 31, 9, 1, 0, 25, 28, 32, 2, 31, 28, 32, 33, 31, 28, 32, 2, 32, 19, 2, 32, 34, 15, 35, 23, 38, 15, 31, 28, 27, 2, 19, 27, 2]
